In [68]:
import pandas as pd
import numpy as np

from os.path import join

In [90]:
INPUT_DIR = '../input'
train_df = pd.read_csv(join(INPUT_DIR, 'train.csv'))
print(f'Number of samples in train: {train_df.shape[0]}')
print('Train dictribution:')
train_df['target'].value_counts(normalize=True) * 100

Number of samples in train: 33126
Train dictribution:


0    98.237034
1     1.762966
Name: target, dtype: float64

In [91]:
SUBMISSIONS_DIR = '../submissions'
exp_name = 'exp_train_02'

In [92]:
submission_df = pd.read_csv(join(SUBMISSIONS_DIR, f'{exp_name}.csv'))
print(f'Number of samples in test: {submission_df.shape[0]}')
print('Predicted test dictribution:')
submission_df['target'].apply(round).value_counts(normalize=True) * 100

Number of samples in test: 10982
Predicted test dictribution:


0    99.480969
1     0.519031
Name: target, dtype: float64

In [72]:
hist, bin_edges = np.histogram(submission_df['target'], [0, .05, .1, .15, .2, .5, .8, .85, .90, .95, 1])

In [73]:
[round(x / len(submission_df) * 100, 3) for x in hist], bin_edges

([91.577, 3.396, 1.539, 1.029, 1.94, 0.474, 0.036, 0.009, 0.0, 0.0],
 array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.5 , 0.8 , 0.85, 0.9 , 0.95, 1.  ]))

In [74]:
hist, bin_edges = np.histogram(submission_df['target'], [0, .5, 1])

In [75]:
[round(x / len(submission_df) * 100, 3) for x in hist], [round(x , 3) for x in hist]

([99.481, 0.519], [10925, 57])

In [76]:
submission_df[submission_df['target'] >= 0.5].sort_values(by='target')

,image_name,target
6730,ISIC_6096565,0.501545
4796,ISIC_4416068,0.504830
9288,ISIC_8455665,0.507174
1786,ISIC_1694285,0.507319
8745,ISIC_7940557,0.507772
1525,ISIC_1460169,0.513063
5243,ISIC_4814164,0.521416
951,ISIC_0945202,0.521758
390,ISIC_0470699,0.530370
9171,ISIC_8345288,0.535157
